In [1]:
import yfinance as yf

In [14]:
sp500 = yf.Ticker("^GSPC")
sp500_history = sp500.history(period="max",actions=False, start="1950-01-01")

In [13]:
first_row = sp500_history[sp500_history["Volume"] != 0].iloc[0] # Firs row with non-zero volume
first_row

Open           16.66
High           16.66
Low            16.66
Close          16.66
Volume    1260000.00
Name: 1950-01-03 00:00:00-05:00, dtype: float64

In [17]:
# get nans
nans = sp500_history.isna().sum()
nans

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [19]:
import numpy as np
# Convert data to numpy array
sp500_data = sp500_history.to_numpy()

(19005, 5)

In [56]:
SEQUENCE_LENGTH = 30

sequences = np.zeros((len(sp500_data) - SEQUENCE_LENGTH, SEQUENCE_LENGTH, sp500_data.shape[1]))
labels = np.zeros((len(sp500_data) - SEQUENCE_LENGTH, 1))


for i in range(SEQUENCE_LENGTH, sp500_data.shape[0]):
    labels[i - SEQUENCE_LENGTH] = sp500_data[i, 3] # Close price
    sequences[i - SEQUENCE_LENGTH] = sp500_data[i-SEQUENCE_LENGTH:i, :] # Open, High, Low, Close, Volume
    

In [36]:
labels.shape, sequences.shape

((18975, 1), (18975, 30, 5))

In [ ]:
# Normalize sequences per sequence to avoid data leakage
close_mean = np.mean(sequences[:, :, 3], axis=1, keepdims=True)
close_std = np.std(sequences[:, :, 3], axis=1, keepdims=True)

volume_mean = np.mean(sequences[:, :, 4], axis=1, keepdims=True)
volume_std = np.std(sequences[:, :, 4], axis=1, keepdims=True)

sequences[:, :, :4] = (sequences[:, :, :4] - close_mean[:, np.newaxis, :]) / close_std[:, np.newaxis, :]
sequences[:, :, 4] = (sequences[:, :, 4] - volume_mean) / volume_std
labels = (labels - close_mean[:, 0]) / close_std[:, 0]  # Normalize labels

((18975, 1), (18975, 1))

In [60]:
sequences[:, :, :4].shape, volume_mean.shape

((18975, 30, 4), (18975, 1))

array([[1.66599998e+01, 1.66599998e+01, 1.66599998e+01, 1.66599998e+01,
        1.26000000e+06],
       [1.68500004e+01, 1.68500004e+01, 1.68500004e+01, 1.68500004e+01,
        1.89000000e+06],
       [1.69300003e+01, 1.69300003e+01, 1.69300003e+01, 1.69300003e+01,
        2.55000000e+06],
       ...,
       [6.29529004e+03, 6.30204004e+03, 6.24168018e+03, 6.24375977e+03,
        5.13367000e+09],
       [6.25450000e+03, 6.26812012e+03, 6.20158984e+03, 6.26370020e+03,
        5.17746000e+09],
       [6.26339990e+03, 6.29612012e+03, 6.26227002e+03, 6.29491016e+03,
        1.70154900e+09]], shape=(19005, 5))

array([[[1.66599998e+01, 1.66599998e+01, 1.66599998e+01, 1.66599998e+01,
         1.26000000e+06],
        [1.68500004e+01, 1.68500004e+01, 1.68500004e+01, 1.68500004e+01,
         1.89000000e+06],
        [1.69300003e+01, 1.69300003e+01, 1.69300003e+01, 1.69300003e+01,
         2.55000000e+06],
        ...,
        [1.72800007e+01, 1.72800007e+01, 1.72800007e+01, 1.72800007e+01,
         1.81000000e+06],
        [1.72399998e+01, 1.72399998e+01, 1.72399998e+01, 1.72399998e+01,
         1.79000000e+06],
        [1.70599995e+01, 1.70599995e+01, 1.70599995e+01, 1.70599995e+01,
         2.21000000e+06]],

       [[1.68500004e+01, 1.68500004e+01, 1.68500004e+01, 1.68500004e+01,
         1.89000000e+06],
        [1.69300003e+01, 1.69300003e+01, 1.69300003e+01, 1.69300003e+01,
         2.55000000e+06],
        [1.69799995e+01, 1.69799995e+01, 1.69799995e+01, 1.69799995e+01,
         2.01000000e+06],
        ...,
        [1.72399998e+01, 1.72399998e+01, 1.72399998e+01, 1.72399998e+01,
        